In [1]:
from pettingzoo.classic import rps_v2
from pettingzoo import AECEnv
from utils import write_to_file, lm, extract_enclosed_text
from pathlib import Path
from prompts.prompt import *
from prompts.rps import *

# you should add your API key to .env file, OPENAI_API_KEY=sk
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Indicate the LLM version that you want to use
GPT_MODEL = 'gpt-3.5-turbo'
# max number of trials for each task
NUM_TRY = 6
# By default, the max number of tokens is 1400. In some tasks, the context limit of the language models is exceeded. Try to change the number of tokens in these cases.
MAX_TOKENS = 1400
# Output path
LOG_PATH = Path('./log/signature/')
F_LM = Path('lm_log.txt')
F_GAME = Path('game_log.txt')
F_LOG = Path('ipy_log.txt')

In [3]:
dummy_agent_code = \
"""
class Agent:
    def __init__(self, env, name):
        self.env = env
        self.name = name
        self.history = {
            "observations": [],
            "actions": []
        }

    def reset(self):
        \"""
        Reset the agent's state.
        \"""
        self.history = {
            "observations": [],
            "actions": []
        }
    
    def observe(self, observation, reward, termination, truncation, info):
        \"""
        Update the agent's state by storing the observation in the history.
        \"""
        self.history["observations"].append(observation)
    
    def act(self):
        \"""
        Choose the action to take based on the current observation.
        In this game, we will choose the action randomly.
        \"""
        action = self.env.action_space(self.name).sample()
        self.history["actions"].append(action)
        return action
    """

In [4]:
def simulate(agents: any, env: AECEnv) -> dict[any, float]:
    env.reset()
    rewards = {agent: 0 for agent in env.possible_agents}
    
    for agent in agents.values():
        agent.reset()

    for agent_name in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()
        rewards[agent_name] += reward
        obs_message = agents[agent_name].observe(
            observation, reward, termination, truncation, info
        )
        if termination or truncation:
            action = None
        else:
            action = agents[agent_name].act()
        print(f"Agen {agent_name} Action {action}")
        env.step(action)
    env.close()
    return rewards

In [5]:
for _ in range(1):
    prompt_get_agent_class = get_solution_prompt\
        .replace("<simulation code>", simulation_code)\
        .replace("<starter>", rps_starter)\
        .replace("<env info>", rps_info)
    write_to_file(LOG_PATH, F_LOG, prompt_get_agent_class)
    response = lm(prompt_get_agent_class, GPT_MODEL, MAX_TOKENS, LOG_PATH, F_LM)
    write_to_file(LOG_PATH, F_LOG, response)
    define_agent_code = extract_enclosed_text(response, "```python", "```")
    # define_agent_code = dummy_agent_code
    env = rps_v2.env(max_cycles=5, render_mode="human")
    Agent = None
    exec(define_agent_code)
    agents = {
        name: Agent(env, name)
        for name in env.possible_agents
    }
    rewards = simulate(agents, env)
    print(rewards)
    # get_refinement_prompt = f"given the execution result: {info}"
    # refinement_response = 
    # (get_refinement_prompt)

Agen player_0 Action 1
Agen player_1 Action 1
Agen player_0 Action 2
Agen player_1 Action 0
Agen player_0 Action 1
Agen player_1 Action 1
Agen player_0 Action 1
Agen player_1 Action 0
Agen player_0 Action 0
Agen player_1 Action 2
Agen player_0 Action None
Agen player_1 Action None
{'player_0': 1, 'player_1': -1}
